##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Basic code generation

This notebook demonstrates how to use prompting to perform basic code generation using the Gemini API's Python SDK. Two use cases are explored: error handling and code generation.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/prompting/Basic_Code_Generation.ipynb"><img src = "../../images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

The Gemini API can be a great tool to save you time during the development process. Tasks such as code generation, debugging, or optimization can be done with the assistance of the Gemini model.

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Additionally, select the model you want to use from the available options below:


In [ ]:
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.0-pro-exp-02-05"] {"allow-input": true, "isTemplate": true}

## Examples

### Error handling

For code generation, we prioritize accuracy over creativity.
A temperature of 0 ensures that the generated content is deterministic,
producing the most sensible output every time.

In [ ]:
from google.genai import types

error_handling_system_prompt =f"""
  Your task is to explain exactly why this error occurred and how to fix it.
"""

error_handling_model_config = types.GenerateContentConfig(
    temperature=0,
    system_instruction=error_handling_system_prompt
)

In [ ]:
from IPython.display import Markdown

error_message = """
    1 my_list = [1,2,3]
  ----> 2 print(my_list[3])

  IndexError: list index out of range
"""

error_prompt = f"""
  You've encountered the following error message:
  Error Message: {error_message}
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=error_prompt,
    config=error_handling_model_config
)

Markdown(response.text)

Okay, let's break down this error and how to resolve it.

**Understanding the Error: `IndexError: list index out of range`**

This error, `IndexError: list index out of range`, is a very common one in Python (and other programming languages). It means you're trying to access an element in a list (or other sequence-like object) using an index that is *outside* the valid range of indices for that list.

*   **Lists are Zero-Indexed:**  In Python, lists are *zero-indexed*. This means the first element in a list is at index 0, the second element is at index 1, the third at index 2, and so on.

*   **Valid Index Range:** For a list of length `n`, the valid indices are from `0` to `n-1`.  Trying to access an element at index `n` or greater (or a negative index that's too far from the end) will raise an `IndexError`.

**Why the Error Occurred in Your Code**

In your code snippet:

```python
my_list = [1, 2, 3]
print(my_list[3])
```

1.  `my_list` is created with three elements: `1`, `2`, and `3`.
2.  The length of `my_list` is 3.
3.  Therefore, the valid indices for `my_list` are 0, 1, and 2.
4.  You're trying to access `my_list[3]`.  Since 3 is *not* a valid index (it's one position beyond the end of the list), Python raises the `IndexError`.

**How to Fix the Error**

To fix the error, you need to make sure you're only accessing elements within the valid index range of the list.  Here are a few common solutions, depending on what you're trying to achieve:

1.  **Access a Valid Element:** If you intended to access the *last* element of the list, you should use index 2:

    ```python
    my_list = [1, 2, 3]
    print(my_list[2])  # Output: 3
    ```

2.  **Use the Length of the List:**  You can dynamically get the index of the last element using `len(my_list) - 1`:

    ```python
    my_list = [1, 2, 3]
    print(my_list[len(my_list) - 1])  # Output: 3
    ```

3.  **Negative Indexing (If Appropriate):** Python allows negative indexing to access elements from the *end* of the list. `my_list[-1]` is the last element, `my_list[-2]` is the second-to-last, and so on.  However, be careful not to go too far negative (e.g., `my_list[-4]` would also cause an `IndexError` in this case).

    ```python
    my_list = [1, 2, 3]
    print(my_list[-1])  # Output: 3
    ```

4.  **Check the List Length Before Accessing:** If you're not sure if the list has enough elements, you can add a check:

    ```python
    my_list = [1, 2, 3]
    index_to_access = 3
    if index_to_access < len(my_list):
        print(my_list[index_to_access])
    else:
        print("Index is out of range.")
    ```

5.  **Looping Through the List:** If you want to process all elements in the list, use a `for` loop:

    ```python
    my_list = [1, 2, 3]
    for item in my_list:
        print(item)
    ```
    Or, if you need the index as well:
     ```python
    my_list = [1, 2, 3]
    for index, item in enumerate(my_list):
        print(f"Element at index {index}: {item}")
    ```

**In Summary**

The `IndexError: list index out of range` error arises when you try to access a list element using an invalid index.  The key is to remember that lists are zero-indexed and to ensure that the index you're using is within the bounds of the list's length.  Choose the appropriate fix based on your intended logic.  Carefully review your code to understand why you're using the specific index that's causing the error.


### Code generation

In [ ]:
code_generation_system_prompt = f"""
  You are a coding assistant. Your task is to generate a code snippet that
  accomplishes a specific goal. The code snippet must be concise, efficient,
  and well-commented for clarity. Consider any constraints or requirements
  provided for the task.

  If the task does not specify a programming language, default to Python.
"""

code_generation_model_config = types.GenerateContentConfig(
    temperature= 0,
    system_instruction=code_generation_system_prompt
  )

In [ ]:
code_generation_prompt = """
  Create a countdown timer that ticks down every second and prints
  "Time is up!" after 20 seconds
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=code_generation_prompt,
    config=code_generation_model_config
)
Markdown(response.text)

```python
import time

def countdown(seconds):
    """
    Counts down from a specified number of seconds, printing the remaining time each second.
    After the countdown reaches zero, it prints "Time is up!".

    Args:
        seconds (int): The number of seconds to count down from.
    """
    for i in range(seconds, 0, -1):
        print(f"{i} seconds remaining")
        time.sleep(1)  # Pause for 1 second
    print("Time is up!")

# Example usage: Countdown from 20 seconds
countdown(20)
```

Let's check if generated code works.

In [ ]:
import time

def countdown(seconds):
  """
  Counts down from a specified number of seconds, printing the remaining time each second.
  After the countdown reaches zero, it prints "Time is up!".

  Args:
    seconds: The number of seconds to count down from.
  """
  for i in range(seconds, 0, -1):
    print(f"{i} seconds remaining")
    time.sleep(1)  # Pause for 1 second
  print("Time is up!")

# Example usage: Countdown from 20 seconds
countdown(20)

20 seconds remaining
19 seconds remaining
18 seconds remaining
17 seconds remaining
16 seconds remaining
15 seconds remaining
14 seconds remaining
13 seconds remaining
12 seconds remaining
11 seconds remaining
10 seconds remaining
9 seconds remaining
8 seconds remaining
7 seconds remaining
6 seconds remaining
5 seconds remaining
4 seconds remaining
3 seconds remaining
2 seconds remaining
1 seconds remaining
Time is up!


## Next steps

Be sure to explore other examples of prompting in the repository. Try writing prompts around your own code as well using the examples in this notebook.